In [1]:
# to delete the cropped images of cattle

import os

parent_directory =  "E:\g42(cattle-proj)\cattle_g42_crop"  # Replace with the path to your "cattle" directory

# Iterate through each subdirectory (representing different cattle)
for cattle_id in os.listdir(parent_directory):
    cattle_directory = os.path.join(parent_directory, cattle_id)

    # Ensure it's a directory
    if os.path.isdir(cattle_directory):
        print(f"Accessing cattle ID: {cattle_id}")

        # Iterate through image files in the cattle's directory
        for image_file in os.listdir(cattle_directory):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                if "muzzle" in image_file.lower():
                    image_path = os.path.join(cattle_directory, image_file)
                    print(f"Deleting image: {image_path}")
                    os.remove(image_path)

# Note: Replace the extensions in `endswith` with the image formats you want to process


Accessing cattle ID: 150_muzzle
Accessing cattle ID: 151_muzzle
Accessing cattle ID: 152_muzzle
Accessing cattle ID: 153_muzzle
Accessing cattle ID: 154_muzzle
Accessing cattle ID: 155_muzzle
Accessing cattle ID: 156_muzzle
Accessing cattle ID: 157_muzzle
Accessing cattle ID: 158_muzzle
Accessing cattle ID: 159_muzzle
Accessing cattle ID: 160_muzzle
Accessing cattle ID: 161_muzzle
Accessing cattle ID: 162_muzzle
Accessing cattle ID: 163_muzzle
Accessing cattle ID: 164_muzzle
Accessing cattle ID: 165_muzzle
Accessing cattle ID: 166_muzzle
Accessing cattle ID: 167_muzzle
Accessing cattle ID: 168_muzzle
Accessing cattle ID: 169_muzzle
Accessing cattle ID: 170_muzzle
Accessing cattle ID: 171_muzzle
Accessing cattle ID: 172_muzzle
Accessing cattle ID: 173_muzzle
Accessing cattle ID: 174_muzzle
Accessing cattle ID: 175_muzzle
Accessing cattle ID: 176_muzzle
Accessing cattle ID: 177_muzzle
Accessing cattle ID: 178_muzzle
Accessing cattle ID: 179_muzzle
Accessing cattle ID: 180_muzzle
Accessin

In [3]:
# to acccess each image present in each cattle id

import os

parent_directory = "E:\g42(cattle-proj)\cattle_g42_crop"  # Replace with the path to your "cattle" directory

# Iterate through each subdirectory (representing different cattle)
for cattle_id in os.listdir(parent_directory):
    cattle_directory = os.path.join(parent_directory, cattle_id)

    # Ensure it's a directory
    if os.path.isdir(cattle_directory):
        print(f"Accessing cattle ID: {cattle_id}")

        # Iterate through image files in the cattle's directory
        for image_file in os.listdir(cattle_directory):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_path = os.path.join(cattle_directory, image_file)
                print(f"Processing image: {image_path}")
                # Here, you can perform any processing you need on the image

# Note: Replace the extensions in `endswith` with the image formats you want to process


Accessing cattle ID: 150_muzzle
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1280.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1282.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1283.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1284.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1285.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1286.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1287.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1289.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1290.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\150_muzzle\IMG_1291.JPG
Accessing cattle ID: 151_muzzle
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\151_muzzle\IMG_1293.JPG
Processing image: E:\g42(cattle-proj)\cattle_g42_crop\151_muzzle\IMG_1294.JPG


In [4]:
import cv2


#!/usr/bin/env python
# coding: utf-8
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader


class YOLO_Pred():
    def __init__(self,onnx_model,data_yaml):
        # load YAML
        with open(data_yaml,mode='r') as f:
            data_yaml = yaml.load(f,Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']
        
        # load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        
    
        
    def predictions(self,image):
        
        row, col, d = image.shape
        # get the YOLO prediction from the the image
        # step-1 convert image into square image (array)
        max_rc = max(row,col)
        input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
        input_image[0:row,0:col] = image
        # step-2: get prediction from square array
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward() # detection or prediction from YOLO

        # Non Maximum Supression
        # step-1: filter detection based on confidence (0.4) and probability score (0.25)
        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        # widht and height of the image (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w/INPUT_WH_YOLO
        y_factor = image_h/INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4] # confidence of detection an object
            if confidence > 0.4:
                class_score = row[5:].max() # maximum probability from 20 objects
                class_id = row[5:].argmax() # get the index position at which max probabilty occur

                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # construct bounding from four values
                    # left, top, width and height
                    left = int((cx - 0.5*w)*x_factor)
                    top = int((cy - 0.5*h)*y_factor)
                    width = int(w*x_factor)
                    height = int(h*y_factor)

                    box = np.array([left,top,width,height])

                    # append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # clean
        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        # NMS
#         index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()
        index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45)
        index = np.array(index).flatten()

        muzzle_detected = False
        
        # Draw the Bounding
        for ind in index:
            # extract bounding box
            x,y,w,h = boxes_np[ind]
            bb_conf = int(confidences_np[ind]*100)
            classes_id = classes[ind]
            class_name = self.labels[classes_id]
            colors = self.generate_colors(classes_id)

            text = f'{class_name}: {bb_conf}%'

            cv2.rectangle(image,(x,y),(x+w,y+h),colors,2)
            cv2.rectangle(image,(x,y-30),(x+w,y),colors,-1)

            cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
            
            if class_name== 'muzzles' or 'muzzle':
                muzzle_detected = True


        if muzzle_detected:
            return image[y:y+h, x:x+w]  # for extracting the only muzzle part  
        else:
            return None
    
    
    def generate_colors(self,ID):
        np.random.seed(10)
        colors = np.random.randint(100,255,size=(self.nc,3)).tolist()
        return tuple(colors[ID])

In [4]:
# yolo = YOLO_Pred('E:\\g42(cattle-proj)\\muzzle_yolo_predictions\\Model10-20231020T053132Z-001\\Model10\\weights\\best.onnx','E:\g42\muzzleCow.v2i.yolov5pytorch\data.yaml')

# def fun(image_path):
#     path = image_path
#     img = cv2.imread(path)
#     if img is None or img.size == 0:
#         print("Invalid or empty image. Cannot process.")
#         os.remove(path)
#         print(f"Deleting image: {path}")
#     else:
#         img = cv2.resize(img, (500,500))
#         img = yolo.predictions(img)  # Save the modified image
        
#         if img is not None:
#             cv2.imwrite(path, img)  # to replace the original image with extracted image
#         else:
#             os.remove(path)
#             print(f"Deleting image: {path}")

In [6]:
yolo = YOLO_Pred('E:\\g42(cattle-proj)\\muzzle_yolo_predictions\\Model10-20231020T053132Z-001\\Model10\\weights\\best.onnx','E:\g42(cattle-proj)\muzzleCow.v2i.yolov5pytorch\data.yaml')

def fun(image_path):
    path = image_path
    img = cv2.imread(path)
    if img is None or img.size == 0:
        print("Invalid or empty image. Cannot process.")
        if os.path.exists(image_path):
            os.remove(path)
            print(f"Deleting image: {path}")
        else:
            exit(0)
    else:
        if img is None or img.size == 0:
            print("Invalid or empty image. Cannot process.")
            exit(0)
        else:
            img = cv2.resize(img, (600,600))
            img = yolo.predictions(img)  # Save the modified image
#             print(img)

            if img is None:
                os.remove(path)
                print(f"Deleting image: {path}")
            elif img.size==0:
                os.remove(path)
                print(f"Deleting image: {path}")
            elif img is not None:
#                 cv2.imshow('prediction image',img)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
                cv2.imwrite(path, img) # to replace the original image with extracted image
            
# fun("E:\g42\cattle_g42_crop\161\IMG_1402.JPG")

In [16]:
list_of_covered_folders=[]

In [17]:
import os

parent_directory = r"E:\g42(cattle-proj)\new_added"  # Replace with the path to your "cattle" directory

# Iterate through each subdirectory (representing different cattle)
for cattle_id in os.listdir(parent_directory):
    cattle_directory = os.path.join(parent_directory, cattle_id)

    # Ensure it's a directory
    if os.path.isdir(cattle_directory):
        if '_muzzle' not in cattle_id and cattle_id not in list_of_covered_folders:
            print(f"Accessing cattle ID: {cattle_id}")
            list_of_covered_folders.append(cattle_id)

            # Iterate through image files in the cattle's directory
            for image_file in os.listdir(cattle_directory):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_path = os.path.join(cattle_directory, image_file)
                    print(f"Processing image: {image_path}")
                    
                    # to replace \ by \\
                    
                    image_path='r'+image_path[0:]
                    i = 0
                    # print(image_path)
                    while i < len(image_path):
                    # print(image_path[i])
                        if image_path[i] == '\\':
                            i = i + 1
                            image_path = image_path[:i] + '\\' + image_path[i:]
                        i = i + 1
                    image_path=image_path[1:]
                    print(image_path)
                    

                    # print(image_path)
                    fun(image_path)
                    
#                     image=cv2.imread(image_path)
#                     image = cv2.resize(image, (300,300))
#                     cv2.imwrite(image_path, yolo.predictions(image))
#                     cv2.imshow('prediction image',yolo.predictions(image))
#                     cv2.waitKey(0)
#                     cv2.destroyAllWindows()


            # Create the new folder name by appending "_muzzle"
            new_cattle_id = f"{cattle_id}_muzzle"
            
            # Rename the folder
            os.rename(cattle_directory, os.path.join(parent_directory, new_cattle_id))
        
        else:
            print("else part",cattle_id)
            
                    # Here, you can perform any processing you need on the image

# Note: Replace the extensions in `endswith` with the image formats you want to process

Accessing cattle ID: 054
Processing image: E:\g42(cattle-proj)\new_added\054\IMG_0390.JPG
E:\\g42(cattle-proj)\\new_added\\054\\IMG_0390.JPG
Deleting image: E:\\g42(cattle-proj)\\new_added\\054\\IMG_0390.JPG


In [18]:
print(list_of_covered_folders)

['054']


In [9]:
# list_of_covered_folders.remove('308')

In [10]:
# to count how many empty folders and non empty folders are present

parent_folder = "E:\g42(cattle-proj)\cattle_g42_crop"  # Replace with the path to your "cattle" folder

# List all subdirectories in the parent folder
subdirectories = [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d))]

# Check each subdirectory for emptiness
empty_folders = []
non_empty_folders = []

for folder in subdirectories:
    folder_path = os.path.join(parent_folder, folder)
    folder_contents = os.listdir(folder_path)
    
    if not folder_contents:
        empty_folders.append(folder)
    else:
        non_empty_folders.append(folder)

print("Empty Folders:")
print("length",len(empty_folders))
for folder in empty_folders:
    print(folder)
    os.remove(path)

print("\nNon-Empty Folders:")
print("length",len(non_empty_folders))
# for folder in non_empty_folders:
#     print(folder)


Empty Folders:
length 0

Non-Empty Folders:
length 146


In [11]:
# to delete empty folders present in the dataset

parent_folder = "E:\g42(cattle-proj)\cattle_g42_crop"  # Replace with the path to your "cattle" folder

# List all subdirectories in the parent folder
subdirectories = [d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d))]

# Delete empty folders
for folder in subdirectories:
    folder_path = os.path.join(parent_folder, folder)
    folder_contents = os.listdir(folder_path)

    if not folder_contents:
        print(f"Deleting empty folder: {folder_path}")
        os.rmdir(folder_path)
